In [125]:
import pandas as pd
import json
import os

In [126]:
bmm = pd.read_csv("_metadados/bedrock_models.csv", sep=";")
pth_mkd_readme = "readme.md"

In [127]:
all_results = pd.DataFrame()

pth = "./results/"
json_files = [pos_json for pos_json in os.listdir(pth) if pos_json.endswith('.json')]

for jf in json_files:
    with open(pth + jf, 'r', encoding='utf-8') as f:
        origin = json.load(f)

        if "results" in origin:
            mx = origin["parameters"]['model']
            c0 = bmm[bmm["Model Id"] == mx].reset_index(drop=True)
            c0i = float(c0["Preço Input 1000"][0])
            c0o = float(c0["Preço Output 1000"][0])
            
            temp = pd.DataFrame(origin["results"])
            temp["eval"] = str(jf).split("_")[1]
            temp['model_id'] = f"{origin["model_desc"]}"
            temp['model_raw'] = origin["parameters"]["model"]
            temp['uses_cot'] = True if origin["parameters"]["cot"] else False
            temp['year'] = origin["parameters"]["year"] if origin["parameters"]["year"] else None
            temp['model_name'] = f"{origin["model_desc"]} - {origin["execution_hash"][0:5]}"
            temp['model_hash'] = origin["execution_hash"]
            temp['cost'] = c0i*1000 + c0o*1000
            temp['real_cost'] = c0i*(temp['input_tokens']/1000)+c0o*(temp['output_tokens']/1000)
            
            all_results = pd.concat([all_results, temp], axis=0)

all_results["correct"] = all_results.apply(lambda row: row["gabarito"] == row["respostas"], axis=1)
all_results["eval_full"] = all_results.apply(lambda row: f"{row["eval"]}/{row["prova_id"] if "prova_id" in row else ""}", axis=1)
all_results.head()

,input_id,gabarito,respostas,respostas_raw,input_tokens,output_tokens,total_tokens,latency,prova_id,eval,model_id,model_raw,uses_cot,year,model_name,model_hash,cost,real_cost,correct,eval_full
0,2016-19_1,D,D,D,374,6,380,850,2016-19,oab,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,us.anthropic.claude-3-5-haiku-20241022-v1:0,False,2016,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,371d456d7da764ca03b5b0845996289d,4.8,0.000323,True,oab/2016-19
1,2016-19_2,B,B,B,409,6,415,684,2016-19,oab,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,us.anthropic.claude-3-5-haiku-20241022-v1:0,False,2016,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,371d456d7da764ca03b5b0845996289d,4.8,0.000351,True,oab/2016-19
2,2016-19_3,C,C,C,561,6,567,784,2016-19,oab,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,us.anthropic.claude-3-5-haiku-20241022-v1:0,False,2016,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,371d456d7da764ca03b5b0845996289d,4.8,0.000473,True,oab/2016-19
3,2016-19_4,C,D,D,422,6,428,1080,2016-19,oab,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,us.anthropic.claude-3-5-haiku-20241022-v1:0,False,2016,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,371d456d7da764ca03b5b0845996289d,4.8,0.000362,False,oab/2016-19
4,2016-19_5,C,C,C,417,6,423,775,2016-19,oab,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,us.anthropic.claude-3-5-haiku-20241022-v1:0,False,2016,us_anthropic_claude-3-5-haiku-20241022-v1:0_ze...,371d456d7da764ca03b5b0845996289d,4.8,0.000358,True,oab/2016-19


In [128]:
benchmark_table = {
    "task": [],
    "hash": [],
    "description": [],
    "provider": [],
    "uses_cot": [],
    "execution_id": [],
    "model": [],
    "accuracy": [],
    "avg_tokens": [],
    "avg_latency": [],
    "latency_per_token": [],
    "cost": [],
    "real_cost_sum": []
}

for task in list(all_results["eval_full"].unique()):
    tdf = all_results[all_results["eval_full"] == task].reset_index(drop=True)
    
    for model in tdf["model_name"].unique():
        m0 = model.split(" - ")[0]
        execution_id = model.split(" - ")[1]
        uses_cot = "cot" in m0.split("_")
        provider = m0.split("_")[1]
        model_idx = m0.split("_")[2]
        hashid = model.split(" - ")[-1]

        benchmark_table["task"].append(task)
        benchmark_table["hash"].append(hashid)
        benchmark_table["description"].append("")
        benchmark_table["provider"].append(provider)
        benchmark_table["uses_cot"].append(uses_cot)
        benchmark_table["execution_id"].append(execution_id)
        benchmark_table["model"].append(f"{model_idx}{" - CoT" if uses_cot else ""}")

        mdf = tdf[tdf["model_name"] == model].reset_index(drop=True)

        benchmark_table["accuracy"].append(round(100*(len(mdf[mdf["correct"]] == True)/len(mdf)),2))
        benchmark_table["avg_tokens"].append(round(mdf["total_tokens"].mean(), 2))
        benchmark_table["avg_latency"].append(round(mdf["latency"].mean(), 2))
        benchmark_table["latency_per_token"].append(round(mdf["latency"].mean()/mdf["total_tokens"].mean(), 2))
        benchmark_table["cost"].append(round(mdf["cost"].mean(), 2))
        benchmark_table["real_cost_sum"].append(round(mdf["real_cost"].sum(), 4))

df_benchmark_table = pd.DataFrame(benchmark_table)

df_benchmark_table['efficiency_index'] = df_benchmark_table['accuracy'].apply(lambda x: x**5)/(10 ** 9)
df_benchmark_table['efficiency_index'] = df_benchmark_table['efficiency_index'] / df_benchmark_table['cost']
df_benchmark_table = df_benchmark_table.sort_values(by=["task", "provider","uses_cot"], ascending=[True, False, True])
df_benchmark_table

,task,hash,description,provider,uses_cot,execution_id,model,accuracy,avg_tokens,avg_latency,latency_per_token,cost,real_cost_sum,efficiency_index
56,enam/enam_2024,08670,,anthropic,False,08670,claude-3-5-sonnet-20241022-v2:0,73.75,520.84,932.41,1.79,18.00,0.1288,0.121210
61,enam/enam_2024,1deb3,,anthropic,False,1deb3,claude-3-5-haiku-20241022-v1:0,37.50,522.19,874.49,1.67,4.80,0.0348,0.015450
59,enam/enam_2024,f8d0f,,anthropic,True,f8d0f,claude-3-5-haiku-20241022-v1:0 - CoT,55.00,810.98,6526.24,8.05,4.80,0.1216,0.104851
55,enam/enam_2024,d64d3,,amazon,False,d64d3,nova-micro-v1:0,33.75,433.55,155.95,0.36,0.18,0.0012,0.243274
60,enam/enam_2024,36cb9,,amazon,False,36cb9,nova-lite-v1:0,30.00,435.12,190.55,0.44,0.30,0.0021,0.081000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,oab/2017-24,bdc8a,,anthropic,True,bdc8a,claude-3-5-haiku-20241022-v1:0 - CoT,68.75,731.95,6355.40,8.68,4.80,0.1149,0.319980
85,oab/2017-24,1a672,,amazon,False,1a672,nova-micro-v1:0,52.50,364.00,141.35,0.39,0.18,0.0010,2.215767
89,oab/2017-24,aeb49,,amazon,False,aeb49,nova-lite-v1:0,62.50,364.00,150.15,0.41,0.30,0.0018,3.178914
90,oab/2017-24,3a6af,,amazon,False,3a6af,nova-pro-v1:0,52.50,364.00,183.10,0.50,4.00,0.0235,0.099709


In [129]:
df_benchmark_table_reduced = df_benchmark_table[df_benchmark_table["model"] != "deepseek-r1-8b-v1:0"]

out = df_benchmark_table_reduced.pivot(index=['task'], columns='model', values='accuracy')
out.loc['AVG']= out.mean(numeric_only=True, axis=0)

print(out.to_markdown())
print(df_benchmark_table.to_markdown())

with open(pth_mkd_readme, mode="r", encoding="utf-8") as f:
    x = f.read()

# Escreve tabela resumo
mkd = x.split("<!-- START - Tabela Resumo -->")[0]
mkd = mkd + "<!-- START - Tabela Resumo -->\n" + out.to_markdown()
mkd = mkd + "\n<!-- END - Tabela Resumo -->" + x.split("<!-- END - Tabela Resumo -->")[1]

mkd = mkd.split("<!-- START - Tabela Detalhada -->")[0]
mkd = mkd + "<!-- START - Tabela Detalhada -->\n" + df_benchmark_table.to_markdown()
mkd = mkd + "\n<!-- END - Tabela Detalhada -->" + x.split("<!-- END - Tabela Detalhada -->")[1]

with open(pth_mkd_readme, mode="w", encoding="utf-8") as f:
    f.write(mkd)

| task           |   claude-3-5-haiku-20241022-v1:0 |   claude-3-5-haiku-20241022-v1:0 - CoT |   claude-3-5-sonnet-20241022-v2:0 |   nova-lite-v1:0 |   nova-lite-v1:0 - CoT |   nova-micro-v1:0 |   nova-micro-v1:0 - CoT |   nova-pro-v1:0 |   nova-pro-v1:0 - CoT |
|:---------------|---------------------------------:|---------------------------------------:|----------------------------------:|-----------------:|-----------------------:|------------------:|------------------------:|----------------:|----------------------:|
| enam/enam_2024 |                          37.5    |                                55      |                            73.75  |          30      |                30      |           33.75   |                 43.75   |         46.25   |                46.25  |
| enem/2022      |                          63.13   |                                81.01   |                            75.98  |          67.04   |                76.54   |           64.8    |                 

In [130]:
import plotly.express as px

plotter = []
temp = out.reset_index()
tasks = list(temp["task"].unique())
models = list(temp.columns)[1:]

min_r = 100
max_r = 0

for t in tasks:
    for m in models:
        tx = temp[temp["task"] == t][m].reset_index(drop=True)[0]
        
        if tx < min_r:
            min_r = tx
        if tx > max_r:
            max_r = tx
        
        plotter.append({"task": t, "model": m, "score": tx})

plotter = pd.DataFrame(plotter)
fig = px.line_polar(plotter, r="score", theta="task", color="model", height=1400, line_close=True, markers=True, range_r=[min_r-5,max_r+5])
fig.write_image("2_charts/radar.png", height = 1400, width = 1400)
# fig.write_image("2_charts/radar.png")
fig.show()


In [131]:
import plotly.express as px

plotter = []
temp = out.reset_index()
temp_enem = temp[temp['task'].isin(["enem/2022","enem/2023","enem/2024"])]

tasks = list(temp_enem["task"].unique())
models = list(temp_enem.columns)[1:]

min_r = 100
max_r = 0

for t in tasks:
    for m in models:
        tx = temp_enem[temp_enem["task"] == t][m].reset_index(drop=True)[0]
        
        if tx < min_r:
            min_r = tx
        if tx > max_r:
            max_r = tx
        
        plotter.append({"task": t, "model": m, "score": tx})

plotter = pd.DataFrame(plotter)
fig = px.line_polar(plotter, r="score", theta="task", color="model", height=1400, line_close=True, markers=True, range_r=[min_r-5,max_r+5])
fig.write_image("2_charts/radar.png", height = 1400, width = 1400)
# fig.write_image("2_charts/radar.png")
fig.show()


In [132]:
enem_e2e = df_benchmark_table[df_benchmark_table['task'].isin(["enem/2022","enem/2023","enem/2024"])]
enem_e2e['base_model'] = enem_e2e['model'].apply(lambda x: x.split(' - ')[0].replace('-v1:0','').replace('-v2:0',''))
enem_e2e = enem_e2e.groupby(['base_model', 'model']).agg({'accuracy': 'mean', 'latency_per_token': 'mean', 'real_cost_sum': 'mean'}).reset_index()
enem_e2e

/var/folders/3v/mw3mqm_n2b790_112gdh2xdc0000gq/T/ipykernel_9935/3465430101.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,base_model,model,accuracy,latency_per_token,real_cost_sum
0,claude-3-5-haiku-20241022,claude-3-5-haiku-20241022-v1:0,62.200000,1.856667,0.064400
1,claude-3-5-haiku-20241022,claude-3-5-haiku-20241022-v1:0 - CoT,83.053333,8.473333,0.256133
2,claude-3-5-sonnet-20241022,claude-3-5-sonnet-20241022-v2:0,79.330000,1.960000,0.247500
3,nova-lite,nova-lite-v1:0,70.950000,0.413333,0.004033
4,nova-lite,nova-lite-v1:0 - CoT,78.586667,3.933333,0.019067
5,nova-micro,nova-micro-v1:0,66.106667,0.373333,0.002367
6,nova-micro,nova-micro-v1:0 - CoT,77.283333,3.176667,0.012267
7,nova-pro,nova-pro-v1:0,75.420000,0.440000,0.053733
8,nova-pro,nova-pro-v1:0 - CoT,85.663333,5.493333,0.204633


In [133]:
px.scatter(enem_e2e, x='real_cost_sum', y='accuracy', size='latency_per_token', color='model', symbol = 'base_model', height=800)